## Week 6 notebook

In [2]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv(r'C:\Users\JT von Seggern\DS Masters Repos\2025-summer-mod-6\homework_6.1.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head()

,Z,X,Y
0,0.548814,0,-0.823220
1,0.715189,1,0.842405
2,0.602763,1,0.898618
3,0.544883,0,-0.817325
4,0.423655,0,-0.635482


### Matching and Treatment Effect Analysis

- Match treated (X=1) to untreated (X=0) based on confounder (Z) using NearestNeighbors.
- Compute the average treatment effect (ATE) by pairing each item with its nearest counterfactual.
- Compute the average treatment effect on the treated (ATT).
- Compute the average treatment effect on the untreated (ATU).
- Compute the marginal treatment effect (maximum effect among untreated).


In [7]:
from sklearn.neighbors import NearestNeighbors

# Assume df has columns: X (treatment), Y (outcome), Z (confounder)
# Separate treated and untreated
T = df[df['X'] == 1].copy()
U = df[df['X'] == 0].copy()

# Fit NearestNeighbors on Z for each group
treated_nn = NearestNeighbors(n_neighbors=1).fit(U[['Z']])
untreated_nn = NearestNeighbors(n_neighbors=1).fit(T[['Z']])

# For each treated, find nearest untreated (counterfactual)
dist_T, idx_T = treated_nn.kneighbors(T[['Z']])
T['cf_idx'] = idx_T.flatten()
T['cf_Y'] = U.iloc[T['cf_idx']]['Y'].values
T['effect'] = T['Y'] - T['cf_Y']

# For each untreated, find nearest treated (counterfactual)
dist_U, idx_U = untreated_nn.kneighbors(U[['Z']])
U['cf_idx'] = idx_U.flatten()
U['cf_Y'] = T.iloc[U['cf_idx']]['Y'].values
U['effect'] = U['cf_Y'] - U['Y']

# Average Treatment Effect (ATE): average of all effects (both directions)
ate = pd.concat([T['effect'], U['effect']]).mean()

# Average Treatment Effect on the Treated (ATT): average effect for treated
att = T['effect'].mean()

# Average Treatment Effect on the Untreated (ATU): average effect for untreated
atu = U['effect'].mean()

# Marginal Treatment Effect: maximum effect among untreated
mte = U['effect'].max()

print(f"ATE: {ate:.3f}")
print(f"ATT: {att:.3f}")
print(f"ATU: {atu:.3f}")
print(f"Marginal Treatment Effect: {mte:.3f}")

ATE: 1.695
ATT: 1.846
ATU: 1.549
Marginal Treatment Effect: 2.172
